In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext import data
from tqdm import tnrange, tqdm_notebook
import torch.nn.functional as F
from data_loader import DataLoader
from torch.autograd import Variable
%load_ext autoreload
%autoreload 2

In [2]:
data_loader = DataLoader()
train, valid = data_loader.small_train_valid()

loading data...
splitting data...
building vocabulary...


In [3]:
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# make iterator for splits
train_iter, valid_iter = data.BucketIterator.splits((train, valid), batch_size=BATCH_SIZE, device=device)

### Model Config

In [4]:
VOCAB_SIZE, EMBEDDING_DIM = data_loader.TEXT.vocab.vectors.shape
KERNEL_NUM = 100
KERNEL_SIZES = [3,4,5]
CLASS_NUM = 1
DROP_OUT = 0.5

### CNN

In [5]:
class CNN_Text(nn.Module):
    
    def __init__(self, vocab_size, embed_dim, class_num, kernel_num, kernel_sizes, drop_out):
        super(CNN_Text, self).__init__()
#         self.args = args
        
#         V = args.embed_num
#         D = args.embed_dim
#         C = args.class_num
#         Ci = 1
#         Co = args.kernel_num
#         Ks = args.kernel_sizes
        
        V = vocab_size
        D = embed_dim
        C = class_num
        Ci = 1
        Co = kernel_num
        Ks = kernel_sizes

        self.embed = nn.Embedding(V, D)
        # self.convs1 = [nn.Conv2d(Ci, Co, (K, D)) for K in Ks]
        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])
        '''
        self.conv13 = nn.Conv2d(Ci, Co, (3, D))
        self.conv14 = nn.Conv2d(Ci, Co, (4, D))
        self.conv15 = nn.Conv2d(Ci, Co, (5, D))
        '''
        self.dropout = nn.Dropout(drop_out)
        self.fc1 = nn.Linear(len(Ks)*Co, C)

    def conv_and_pool(self, x, conv):
        x = F.relu(conv(x)).squeeze(3)  # (N, Co, W)
        x = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x

    def forward(self, x):
        x = self.embed(x)  # (N, W, D)
        
#         if self.args.static:
#             x = Variable(x)
        x = x.unsqueeze(1)  # (N, Ci, W, D)

        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)

        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)

        x = torch.cat(x, 1)

        '''
        x1 = self.conv_and_pool(x,self.conv13) #(N,Co)
        x2 = self.conv_and_pool(x,self.conv14) #(N,Co)
        x3 = self.conv_and_pool(x,self.conv15) #(N,Co)
        x = torch.cat((x1, x2, x3), 1) # (N,len(Ks)*Co)
        '''
        x = self.dropout(x)  # (N, len(Ks)*Co)
        logit = self.fc1(x)  # (N, C)
        return logit

In [6]:
# initialize model
model_cnn = CNN_Text(VOCAB_SIZE, EMBEDDING_DIM, CLASS_NUM, KERNEL_NUM, KERNEL_SIZES, DROP_OUT)
model_cnn = model_cnn.to(device)  # place it to GPU (if available)

# replace inintial weights of embedding layer with pred-trained embedding
pretrained_embeddings = data_loader.TEXT.vocab.vectors
model_cnn.embed.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.6815,  0.5909, -0.7525,  ...,  0.3713, -0.2969,  0.0298],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2415, -0.1050, -0.1875,  ..., -0.3229,  0.1251, -0.8694]])

In [7]:
class SentimentClassifier:
    
    def __init__(self, train_iter, valid_iter, model):
        self.train_iter = train_iter
        self.valid_iter = valid_iter
        self.model = model
        self.optimizer = optim.Adam(self.model.parameters())
        self.loss_function = nn.BCEWithLogitsLoss()          # Binary Cross-Entropy Loss
        self.loss_function = self.loss_function.to(device)   # place it to GPU (if available)
        
    def accuracy(self, pred, y):
        """Define metric for evaluation."""
        pred = torch.round(torch.sigmoid(pred))
        acc = torch.sum((pred == y)).float() / len(y)
        return acc

    def train_model(self):
        """Train one epoch of inputs and update weights.
        
        Return: average loss, average accuracy.
        """
        epoch_loss = []
        epoch_acc = []
        self.model.train()

        for batch_data in self.train_iter:
            self.optimizer.zero_grad()  # clear out gradient
            pred = self.model(batch_data.text.t_()).squeeze(1)
            y = (batch_data.label.squeeze(0) >= 3).float()  # neg:2, pos:3 -> convert them to 0 and 1
            loss = self.loss_function(pred, y)
            acc = self.accuracy(pred, y)

            # backprob and update gradient
            loss.backward()
            self.optimizer.step()

            epoch_loss.append(loss.item())
            epoch_acc.append(acc.item())

        return np.mean(epoch_loss), np.mean(epoch_acc)

    
    def evaluate_model(self):
        """Evaluate one epoch of inputs.
        
        Return: average loss, average accuracy.
        """
        epoch_loss = []
        epoch_acc = []
        self.model.eval()

        with torch.no_grad():    
            for batch_data in self.valid_iter:
                pred = self.model(batch_data.text.t_()).squeeze(1)
                y = (batch_data.label.squeeze(0) >= 3).float()
                loss = self.loss_function(pred, y)
                acc = self.accuracy(pred, y)

                epoch_loss.append(loss.item())
                epoch_acc.append(acc.item())

        return np.mean(epoch_loss), np.mean(epoch_acc)

    
    def run_epochs(self, num_epochs=20, eval_each=1):
        """Run # epochs and evaluate the model.
        
        Return: average loss and accuracy per epoch for training and validation set.
        """
        train_epoch_metrics, valid_epoch_metrics = [], []
        
        for epoch in tnrange(num_epochs, desc='EPOCHS'):
            train_loss, train_acc = self.train_model()
            valid_loss, valid_acc = self.evaluate_model()
            train_epoch_metrics.append((train_loss, train_acc))
            valid_epoch_metrics.append((valid_loss, valid_acc))

            if (epoch + 1) % eval_each == 0:
                print('Epoch %d | Train Loss: %.2f | Train Acc: %.2f | Test Loss: %.2f | Test Acc: %.2f'
                      %(epoch, train_loss, train_acc, valid_loss, valid_acc))
        
        return train_epoch_metrics, valid_epoch_metrics
    
    def save_model(self, PATH):
        torch.save(self.model.state_dict(), PATH)
        
    def load_model(self, PATH):
        self.model.load_state_dict(torch.load(PATH))
        self.model.eval()

In [8]:
cnn_classifier = SentimentClassifier(train_iter, valid_iter, model_cnn)
cnn_train_epoch_metrics, cnn_valid_epoch_metrics = cnn_classifier.run_epochs()

HBox(children=(IntProgress(value=0, description='EPOCHS', max=20, style=ProgressStyle(description_width='initial')), HTML(value='')))

Epoch 0 | Train Loss: 0.71 | Train Acc: 0.52 | Test Loss: 0.68 | Test Acc: 0.50
Epoch 1 | Train Loss: 0.62 | Train Acc: 0.68 | Test Loss: 0.65 | Test Acc: 0.62
Epoch 2 | Train Loss: 0.57 | Train Acc: 0.75 | Test Loss: 0.61 | Test Acc: 0.75
Epoch 3 | Train Loss: 0.50 | Train Acc: 0.81 | Test Loss: 0.57 | Test Acc: 0.77
Epoch 4 | Train Loss: 0.41 | Train Acc: 0.88 | Test Loss: 0.57 | Test Acc: 0.67
Epoch 5 | Train Loss: 0.35 | Train Acc: 0.90 | Test Loss: 0.48 | Test Acc: 0.80
Epoch 6 | Train Loss: 0.27 | Train Acc: 0.94 | Test Loss: 0.46 | Test Acc: 0.79
Epoch 7 | Train Loss: 0.22 | Train Acc: 0.95 | Test Loss: 0.43 | Test Acc: 0.81
Epoch 8 | Train Loss: 0.17 | Train Acc: 0.98 | Test Loss: 0.42 | Test Acc: 0.81
Epoch 9 | Train Loss: 0.12 | Train Acc: 0.99 | Test Loss: 0.41 | Test Acc: 0.82
Epoch 10 | Train Loss: 0.09 | Train Acc: 0.99 | Test Loss: 0.42 | Test Acc: 0.82
Epoch 11 | Train Loss: 0.07 | Train Acc: 0.99 | Test Loss: 0.41 | Test Acc: 0.82
Epoch 12 | Train Loss: 0.06 | Train Ac

In [9]:
cnn_classifier.save_model('cnn-0.pt')

In [10]:
print("Model's state_dict:")
for param_tensor in cnn_classifier.model.state_dict():
    print(param_tensor, "\t", cnn_classifier.model.state_dict()[param_tensor].size())

Model's state_dict:
embed.weight 	 torch.Size([22653, 100])
convs1.0.weight 	 torch.Size([100, 1, 3, 100])
convs1.0.bias 	 torch.Size([100])
convs1.1.weight 	 torch.Size([100, 1, 4, 100])
convs1.1.bias 	 torch.Size([100])
convs1.2.weight 	 torch.Size([100, 1, 5, 100])
convs1.2.bias 	 torch.Size([100])
fc1.weight 	 torch.Size([1, 300])
fc1.bias 	 torch.Size([1])


### Experiment

In [11]:
# train_iter, valid_iter = data.BucketIterator.splits((train, valid), batch_size=len(train))

In [12]:
# train_iter

In [13]:
# for batch in train_iter:
#     print(batch)
#     break

In [14]:
# batch.text.shape

In [15]:
# batch.label.shape

In [16]:
# len([data_loader.TEXT.vocab.itos[i] for i in batch.text[:, 0]])

###### 